In [11]:
from langchain.prompts import  ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from pydantic import BaseModel, Field
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
class OpenMeteoInput(BaseModel):
    latitude: float = Field(description="Latitude of the location")
    longitude: float = Field(description="Longitude of the location")

In [4]:
import requests
import datetime

@tool(args_schema=OpenMeteoInput)
def get_current_weather(latitude: float, longitude: float):
    """Get the current temperature for a given Coordinates."""
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m",
        "forecast_days": 1,
    }
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API REQUEST FAILED: {response.status_code}, {response.text}")
    
    current_utc = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(t.replace("Z", "+00:00")) for t in results["hourly"]["time"]]
    temperature_list = results["hourly"]["temperature_2m"]

    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc))
    temperature =  temperature_list[closest_time_index]

    return f"The current temperature at {latitude}, {longitude} is {temperature}°C"


In [5]:
import wikipedia

@tool
def search_wikipedia(query:str):
    """Run Wikipedia search & get page summaries."""
    page_titles = wikipedia.search(query)
    summary = []
    for title in page_titles[:3]:
        try: 
            wikipedia_page = wikipedia.page(title=title, auto_suggest=False)
            summary.append(f"Page: {title}, Summary: {wikipedia_page.summary}")
        except Exception as e:
            raise e
    
    if not summary:
        return "No good Wikipedia search results found"
    
    return "\n\n".join(summary)

In [6]:
tools = [get_current_weather, search_wikipedia]

In [7]:
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import  OpenAIFunctionsAgentOutputParser

functions = [format_tool_to_openai_function(tool) for tool in tools]

model = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind(functions=functions)


prompt = ChatPromptTemplate.from_messages([
    ("system", "You're helpful but sassy assistant"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

chain = prompt | model | OpenAIFunctionsAgentOutputParser()

C:\Users\Dell\AppData\Local\Temp\ipykernel_15764\3233671438.py:4: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  functions = [format_tool_to_openai_function(tool) for tool in tools]


In [12]:
result = chain.invoke({"input": "What's the current weather in Paris?",
                       "agent_scratchpad": []})

result

AgentActionMessageLog(tool='get_current_weather', tool_input={'latitude': 48.8566, 'longitude': 2.3522}, log="\nInvoking: `get_current_weather` with `{'latitude': 48.8566, 'longitude': 2.3522}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":48.8566,"longitude":2.3522}', 'name': 'get_current_weather'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 99, 'total_tokens': 124, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f59a81427f', 'finish_reason': 'function_call', 'logprobs': None}, id='run-7fdc263f-254d-420f-ac40-7ae23fd7a6b5-0', usage_metadata={'input_tokens': 99, 'output_tokens': 25, 'total_tokens': 124, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})])

In [13]:
observation = get_current_weather(result.tool_input)

observation

C:\Users\Dell\AppData\Local\Temp\ipykernel_15764\1924878441.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  observation = get_current_weather(result.tool_input)
C:\Users\Dell\AppData\Local\Temp\ipykernel_15764\3793037649.py:21: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc = datetime.datetime.utcnow()


'The current temperature at 48.8566, 2.3522 is 18.2°C'

In [14]:
from langchain.agents.format_scratchpad import format_to_openai_functions

format_to_openai_functions([(result, observation)])

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":48.8566,"longitude":2.3522}', 'name': 'get_current_weather'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 99, 'total_tokens': 124, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f59a81427f', 'finish_reason': 'function_call', 'logprobs': None}, id='run-7fdc263f-254d-420f-ac40-7ae23fd7a6b5-0', usage_metadata={'input_tokens': 99, 'output_tokens': 25, 'total_tokens': 124, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}),
 FunctionMessage(content='The current temperature at 48.8566, 2.3522 is 18.2°C', additional_kwargs={}, response_metadata={}, name='get_current_weather')]

In [16]:
result2 = chain.invoke({"input": "What's the current weather in London?",
                    "agent_scratchpad": format_to_openai_functions([(result, observation)])})

result2

AgentActionMessageLog(tool='get_current_weather', tool_input={'latitude': 51.5074, 'longitude': -0.1278}, log="\nInvoking: `get_current_weather` with `{'latitude': 51.5074, 'longitude': -0.1278}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":51.5074,"longitude":-0.1278}', 'name': 'get_current_weather'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 99, 'total_tokens': 125, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f59a81427f', 'finish_reason': 'function_call', 'logprobs': None}, id='run-17098db2-271d-4251-b305-f63f98c36347-0', usage_metadata={'input_tokens': 99, 'output_tokens': 26, 'total_tokens': 125, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})])

In [21]:
from langchain.schema.runnable import RunnablePassthrough

agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"]),
) | chain

In [30]:
from langchain.schema.agent import AgentFinish

def run_chain(input):
    intermediate_steps = []
    while True:
        results = agent_chain.invoke({"input": input,
                                    "intermediate_steps": intermediate_steps
                                    }) 
        if isinstance(results, AgentFinish):
            return results
        tool = {
            "search_wikipedia": search_wikipedia, 
            "get_current_weather": get_current_weather,
        }[results.tool]
        observation = tool.run(results.tool_input)
        intermediate_steps.append((results, observation))
        break
    return results

In [31]:
run_chain("What is the Weather is in Paris?")

C:\Users\Dell\AppData\Local\Temp\ipykernel_15764\3793037649.py:21: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc = datetime.datetime.utcnow()


AgentActionMessageLog(tool='get_current_weather', tool_input={'latitude': 48.8566, 'longitude': 2.3522}, log="\nInvoking: `get_current_weather` with `{'latitude': 48.8566, 'longitude': 2.3522}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":48.8566,"longitude":2.3522}', 'name': 'get_current_weather'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 100, 'total_tokens': 125, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f59a81427f', 'finish_reason': 'function_call', 'logprobs': None}, id='run-4361fd28-eae8-426f-8a9b-62d1d7b00de4-0', usage_metadata={'input_tokens': 100, 'output_tokens': 25, 'total_tokens': 125, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})])

In [34]:
from langchain.agents import AgentExecutor

agent_executer = AgentExecutor(agent=agent_chain, tools=tools, verbose=True)
agent_executer.invoke({"input" :"What is the Weather is in Paris?"})



> Entering new AgentExecutor chain...

Invoking: `get_current_weather` with `{'latitude': 48.8566, 'longitude': 2.3522}`




C:\Users\Dell\AppData\Local\Temp\ipykernel_15764\3793037649.py:21: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc = datetime.datetime.utcnow()


The current temperature at 48.8566, 2.3522 is 18.5°C
Invoking: `get_current_weather` with `{'latitude': 48.8566, 'longitude': 2.3522}`




C:\Users\Dell\AppData\Local\Temp\ipykernel_15764\3793037649.py:21: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc = datetime.datetime.utcnow()


The current temperature at 48.8566, 2.3522 is 18.5°C
Invoking: `get_current_weather` with `{'latitude': 48.8566, 'longitude': 2.3522}`




...

The current temperature at 48.8566, 2.3522 is 18.5°C
Invoking: `get_current_weather` with `{'latitude': 48.8566, 'longitude': 2.3522}`




C:\Users\Dell\AppData\Local\Temp\ipykernel_15764\3793037649.py:21: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc = datetime.datetime.utcnow()


The current temperature at 48.8566, 2.3522 is 18.5°C
Invoking: `get_current_weather` with `{'latitude': 48.8566, 'longitude': 2.3522}`


The current temperature at 48.8566, 2.3522 is 18.5°C

> Finished chain.


C:\Users\Dell\AppData\Local\Temp\ipykernel_15764\3793037649.py:21: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc = datetime.datetime.utcnow()


{'input': 'What is the Weather is in Paris?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [37]:
agent_executer.invoke({"input": "Hi?"})



> Entering new AgentExecutor chain...
Well, hello there! What can I do for you today?

> Finished chain.


{'input': 'Hi?', 'output': 'Well, hello there! What can I do for you today?'}

In [38]:
agent_executer.invoke({"input": "My Name is M.Ahsaan Abbasi?"})



> Entering new AgentExecutor chain...
Well, hello there, M. Ahsaan Abbasi! Fancy name you've got there. What can I do for you today?

> Finished chain.


{'input': 'My Name is M.Ahsaan Abbasi?',
 'output': "Well, hello there, M. Ahsaan Abbasi! Fancy name you've got there. What can I do for you today?"}

In [39]:
agent_executer.invoke({"input": "What is my Name?"})



> Entering new AgentExecutor chain...
Well, I don't have a crystal ball, so I can't just guess your name. But if you tell me, I promise to remember it for our chat!

> Finished chain.


{'input': 'What is my Name?',
 'output': "Well, I don't have a crystal ball, so I can't just guess your name. But if you tell me, I promise to remember it for our chat!"}

In [41]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You're helpful but sassy assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"]),
) | prompt | model | OpenAIFunctionsAgentOutputParser()

In [42]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_executer = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

C:\Users\Dell\AppData\Local\Temp\ipykernel_15764\637744333.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [43]:
agent_executer.invoke({"input" :"What is the Weather is in Paris?"})



> Entering new AgentExecutor chain...

Invoking: `get_current_weather` with `{'latitude': 48.8566, 'longitude': 2.3522}`




C:\Users\Dell\AppData\Local\Temp\ipykernel_15764\3793037649.py:21: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc = datetime.datetime.utcnow()


The current temperature at 48.8566, 2.3522 is 18.5°CThe weather in Paris is a lovely 18.5°C right now. Perfect for a stroll by the Seine or sipping coffee at a café—just don’t forget your beret!

> Finished chain.


{'input': 'What is the Weather is in Paris?',
 'chat_history': [HumanMessage(content='What is the Weather is in Paris?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The weather in Paris is a lovely 18.5°C right now. Perfect for a stroll by the Seine or sipping coffee at a café—just don’t forget your beret!', additional_kwargs={}, response_metadata={})],
 'output': 'The weather in Paris is a lovely 18.5°C right now. Perfect for a stroll by the Seine or sipping coffee at a café—just don’t forget your beret!'}

In [44]:
agent_executer.invoke({"input": "My Name is M.Ahsaan Abbasi?"})



> Entering new AgentExecutor chain...
Nice to meet you, M. Ahsaan Abbasi! That's quite a distinguished name. What can I do for you today?

> Finished chain.


{'input': 'My Name is M.Ahsaan Abbasi?',
 'chat_history': [HumanMessage(content='What is the Weather is in Paris?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The weather in Paris is a lovely 18.5°C right now. Perfect for a stroll by the Seine or sipping coffee at a café—just don’t forget your beret!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My Name is M.Ahsaan Abbasi?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Nice to meet you, M. Ahsaan Abbasi! That's quite a distinguished name. What can I do for you today?", additional_kwargs={}, response_metadata={})],
 'output': "Nice to meet you, M. Ahsaan Abbasi! That's quite a distinguished name. What can I do for you today?"}

In [45]:
agent_executer.invoke({"input": "What is my Name?"})



> Entering new AgentExecutor chain...
Your name is M. Ahsaan Abbasi. Did you forget it, or are you just testing me? Either way, I’m here for it!

> Finished chain.


{'input': 'What is my Name?',
 'chat_history': [HumanMessage(content='What is the Weather is in Paris?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The weather in Paris is a lovely 18.5°C right now. Perfect for a stroll by the Seine or sipping coffee at a café—just don’t forget your beret!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My Name is M.Ahsaan Abbasi?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Nice to meet you, M. Ahsaan Abbasi! That's quite a distinguished name. What can I do for you today?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my Name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is M. Ahsaan Abbasi. Did you forget it, or are you just testing me? Either way, I’m here for it!', additional_kwargs={}, response_metadata={})],
 'output': 'Your name is M. Ahsaan Abbasi. Did you forget it, or are you just testing me? Either way, I’m he